## <font color='Blue'>Introduction</font>
###  In this exercise, we run simulation in a 2D single layer geothermal model.
* The realistic formation heterogeneity is used for permeability map. 
* The simulation can be run under both low- and high-enthalpy conditions with cold water injection.


## <font color='blue'>The objectives:</font>
1. Introduce custom <font color='red'>'Model'</font> class based on 
 * Class [DartsModel](darts/models/darts_model.py) with base model capabilities
 * Class [StructReservoir](darts/models/reservoirs/struct_reservoir.py) with structured reservoir
 * Class [Geothermal](darts/models/physics/geothermal.py) for basic geothermal physics
 * Class [Geothermal_operators](darts/models/physics/geothermal_operators.py) defines OBL operators
2. Use run_python procedure to control run from the python script
3. Introduce wells and time-dependent well controls
4. Redefine physical properties and run simulation with custom-defined property.


## <font color='Blue'>Let's start!</font>

### 1. Import the physics, base model and simulation model into workspace

In [ ]:
from darts.models.darts_model import DartsModel
from darts.models.reservoirs.struct_reservoir import StructReservoir
from darts.models.physics.geothermal import Geothermal
from darts.engines import value_vector
import numpy as np

### 2. Build Model class
#### Brief Introduction of model inheritance
* Here create the <font color='red'>'Model' </font>  class, which inherits from <font color='red'>DartsModel</font> (the base class).
* It keeps all the functionalities of <font color='red'>DartsModel</font> and can also be extended to add more functionalities.
* If a function is redefined in subclass, the function in base class with identical name will be overridden (just like C++).

In [ ]:
class Model(DartsModel):
    def __init__(self):
        # call base class constructor
        super().__init__()

        self.timer.node["initialization"].start()
        
        # predefined reservoir size: 60*40*1
        self.nx = 60
        self.ny = 40
        self.nz = 1
        # read-in permeability and porosity from files
        self.kx = np.genfromtxt('Geothermal/permx_2D.txt', skip_header=True, skip_footer=True).flatten()
        self.ky = np.genfromtxt('Geothermal/permy_2D.txt', skip_header=True, skip_footer=True).flatten()
        self.kz = np.genfromtxt('Geothermal/permz_2D.txt', skip_header=True, skip_footer=True).flatten()
        self.poro = np.genfromtxt('Geothermal/poro_2D.txt', skip_header=True, skip_footer=True).flatten()
        
        # Create reservoir using StructReservoir. 
        # Just pass-in the necessary parameters and a box reservoir is generated.
        self.reservoir = StructReservoir(self.timer, nx=self.nx, ny=self.ny, nz=self.nz, dx=30, dy=30,
                                         dz=2.5, permx=self.kx, permy=self.ky, permz=self.kz,
                                         poro=self.poro, depth=1000)
        # Get the number of reservoir grids
        self.nb = self.reservoir.mesh.n_res_blocks
        
        # Create numpy arrays wrapped around mesh data (no copying)
        self.volume = np.array(self.reservoir.mesh.volume, copy=False)
        self.hcap = np.array(self.reservoir.mesh.heat_capacity, copy=False)
        self.cond = np.array(self.reservoir.mesh.rock_cond, copy=False)

        # Set open-flow boundary condition
        self.volume[0:self.nb:60] = 8e9
        self.volume[59:self.nb:60] = 8e9
        
        # Constant definitions
        self.cond.fill(200)
        self.hcap.fill(2200)

        # add wells        
        self.reservoir.add_well("I1")
        self.reservoir.add_perforation(well=self.reservoir.wells[-1], i=15, j=20, k=1, well_index=10,
                                           multi_segment=False, verbose=True)

        self.reservoir.add_well("P1")
        self.reservoir.add_perforation(well=self.reservoir.wells[-1], i=48, j=20, k=1, well_index=10,
                                           multi_segment=False, verbose=True)
        
        # add predefined physics
        self.physics = Geothermal(self.timer, 128, 0, 500, 1000, 20000, grav=True)
        
        # time step setting
        self.params.first_ts = 0.001
        self.params.mult_ts = 2
        self.params.max_ts = 100

        # Newton tolerance is relatively high because of L2-norm for residual and well segments
        self.params.tolerance_newton = 1e-3
        self.params.tolerance_linear = 1e-5
        self.params.max_i_newton = 20
        self.params.max_i_linear = 50
        
        # default runtime
        self.runtime = 1000
        self.timer.node["initialization"].stop()
        
    '''Give initial pressure and temperature conditions to reservoir'''
    def set_initial_conditions(self):
        self.physics.set_uniform_initial_conditions(self.reservoir.mesh, uniform_pressure=200, \
                                                    uniform_temperature=348.15)
        
    '''Give the well controls'''
    def set_boundary_conditions(self):
        for i, w in enumerate(self.reservoir.wells):
            if i == 0:
                w.control = self.physics.new_bhp_water_inj(250, 308.15)
            else:
                w.control = self.physics.new_bhp_prod(150)
                              

### <font color='red'>Note: The following correction should be added to class above</font>

<img style="float: left;" src="slides/Slide27.JPG" width="60%">

### 3. Create and initialize simulation model

In [ ]:
# build a Model object
m = Model()
m.init()

### 4. Run simulation and print statistics

In [ ]:
m.run_python()
m.print_timers()

### 5. Data process and plot

In [ ]:
%matplotlib inline

import pandas as pd
# access to engine time-dependent data
time_data = pd.DataFrame.from_dict(m.physics.engine.time_data)
# wirte timedata to output file
time_data.to_pickle("darts_time_data.pkl")
# write timedata to excel file
writer = pd.ExcelWriter('time_data.xlsx')
time_data.to_excel(writer, 'Sheet1')
writer.save()

# read data from output file
time_data = pd.read_pickle("darts_time_data.pkl")

from darts.tools.plot_darts import *
# plot production temperature
p_w = 'P1'
ax = plot_water_rate_darts(p_w, time_data)
ax = plot_temp_darts(p_w, time_data)
plt.show()

In [ ]:
# access to engine solution
X = np.array(m.physics.engine.X, copy=False)
# plot pressure map and surface, layer 1
m.plot_layer_map_offline(X[0::2], 1, "Pressure")

In [ ]:
# plot temperature map and surface, layer 1
from darts.models.physics.iapws.iapws_property import iapws_temperature_evaluator
# evaluator of temperature from property package
temperature = iapws_temperature_evaluator()
data_len = int(len(X) / 2)
Temp = np.zeros(data_len)
# compute temperature block-wise
for i in range(0, data_len):
    Temp[i] = temperature.evaluate([X[2 * i], X[2 * i + 1]])
# plot temperatu
m.plot_layer_map_offline(Temp, 1, "Temperature")

<img style="float: left;" src="slides/Slide28.JPG" width="60%">

<img style="float: left;" src="slides/Slide29.JPG" width="60%">

<img style="float: left;" src="slides/Slide30.JPG" width="60%">

<img style="float: left;" src="slides/Slide31.JPG" width="100%">